In [ ]:
import gurobipy as gp
import pandas as pd

# Data Manipulation
file = "fake_claims_data.csv"
csv_data = pd.read_csv(file, header = None)


# Parameters

a = 0.01 # Percentage of total radiologists in Sole practices
b = 0.10 # Percentage of total radiologists in Small practices
c = 0.51 # Percentage of total radiologists in Medium practices
d = 0.38 # Percentage of total radiologists in Large practices

n = 46 # Number of Sole practices 
m = 108 # Number of Small practices
o = 79 # Number of Medium practices
p = 9 # Number of Large practices

s = n+m+o+p # Number of Total practices
r = len(csv_data) # Number of Total Cases

# Decision Variables

model = gp.Model('Radiology Case Allocation') # Creating Model Instance 
model.Params.IntegralityFocus = 1


j_index = [j for j in range(1,s+1)]
n_index = [n for n in range(1,n+1)]
m_index = [m for m in range(n+1,n+m+1)]
o_index = [o for o in range(n+m+1,s-p+1)]
p_index = [p for p in range(s-p+1,s+1)]

ijk_index = [] # Creating indices for Variable Xijk
for case in range(1,r+1):
    for prac in range(1,s+1):
        if 1 <= prac <= n:
            ijk_index.append((case,prac,1))
        if n < prac <= n+m:
            ijk_index.append((case,prac,2))
        if n+m < prac <= n+m+o:
            ijk_index.append((case,prac,3))
        if n+m+o < prac:
            ijk_index.append((case,prac,4))

x = model.addVars(ijk_index, name="x_",vtype=gp.GRB.BINARY) 
z1 = model.addVar(name="z1", vtype= gp.GRB.CONTINUOUS)
z2 = model.addVar(name="z2", vtype= gp.GRB.CONTINUOUS)
z3 = model.addVar(name="z3", vtype= gp.GRB.CONTINUOUS)
z4 = model.addVar(name="z4", vtype= gp.GRB.CONTINUOUS)


# Objective Function
model.setObjective(z1+z2+z3+z4, gp.GRB.MINIMIZE)

# Constraints

# 1st Constraint: 'Each practice has at least 1 case'
model.addConstrs((x.sum('*',j) >= 1 for j in j_index),name="1")

# 2nd Constraint: 'Every practice in the 4-bin practice category version (i.e. sole, small, medium, and large)
# has the same number of cases in each category'
model.addConstrs((0 == x.sum('*','*',1)/n - x.sum('*',j,1)  for j in n_index),name="2a")
model.addConstrs((0 == x.sum('*','*',2)/m - x.sum('*',j,2)  for j in m_index),name="2b")
model.addConstrs((0 == x.sum('*','*',3)/o - x.sum('*',j,3)  for j in o_index),name="2c")
model.addConstrs((0 == x.sum('*','*',4)/p - x.sum('*',j,4) for j in p_index),name="2d")

# 3rd Constraint: 'The sum of cases in each 4-bin category (i.e. sole, small, medium, and large)
# matches the distribution of the “total % of radiologists” in the 4-bin categories'
model.addConstr((z1 >= (x.sum('*','*',1)/r) - a),name="3a")
model.addConstr((z1 >= (-x.sum('*','*',1)/r) + a),name="3b")
model.addConstr((z2 >= (x.sum('*','*',2)/r) - b),name="3c")
model.addConstr((z2 >= (-x.sum('*','*',2)/r) + b),name="3d")
model.addConstr((z3 >= (x.sum('*','*',3)/r) - c),name="3e")
model.addConstr((z3 >= (-x.sum('*','*',3)/r) + c),name="3f")
model.addConstr((z4 >= (x.sum('*','*',4)/r) - d),name="3g")
model.addConstr((z4 >= (-x.sum('*','*',4)/r) + d),name="3h")

# 4th Constraint: 'Every Case is Allocated'
model.addConstr((x.sum('*','*','*') == r), name="4")

# Run Model
model.optimize()

# Case Allocation
Allocated_Cases = [var.x for var in x.values()]



Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads
Optimize a model with 493 rows, 1007692 columns and 87735488 nonzeros
Model fingerprint: 0x1dc6ce8d
Variable types: 4 continuous, 1007688 integer (1007688 binary)
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 4e+03]
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "c:\program files\python36\lib\site-packages\ipykernel\iostream.py", line 384, in write
    def write(self, string):
KeyboardInterrupt


Presolve removed 239 rows and 1007685 columns (presolve time = 207s) ...
Presolve removed 486 rows and 1007685 columns
Presolve time: 207.57s
Presolved: 7 rows, 7 columns, 16 nonzeros
Variable types: 3 continuous, 4 integer (0 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0470701e-03   1.235054e+02   0.000000e+00    220s
       4    2.0941402e-03   0.000000e+00   0.000000e+00    220s

Root relaxation: objective 2.094140e-03, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00209    0    3          -    0.00209      -     -  219s
H    0     0                       0.1245533    0.00209  98.3%     -  219s
H    0     0                       0.0899712    0.00209  97.7%     -  219s
     0     0    0.01767    0    4    0.08997    0.01767  80.4%     -  219s
     0     0    0.01799    0  

In [2]:
# This piece of code is checking to make sure all cases were allocated.

Variable_Values = list(zip(ijk_index,Allocated_Cases))
Actual_Values = []
for items in Variable_Values:
    if items[1] == 1:
        Actual_Values.append(items)
print(len(Actual_Values))

4078


In [11]:
# This piece of code is creating the lists of four Radiology practice groups

Practice_Case = []
for num in range(1,s+1):
    case_count = 0
    practice_type = 0
    for item in Actual_Values:
        if item[0][1] == num:
            case_count += 1
            practice_type = item[0][2]
    Practice_Case.append((num,case_count,practice_type))  


In [12]:
# This piece of code is checking to ensure that the four Radiology practice groups have the correct percentages
# of total cases

solo = 0
small = 0
medium = 0
large = 0

for items in Practice_Case:
    if items[2] == 1:
        solo += items[1]
    elif items[2] == 2:
        small += items[1]
    elif items[2] == 3:
        medium += items[1]
    elif items[2] == 4:
        large += items[1]
print(solo/4164)
print(small/4164)
print(medium/4164)
print(large/4164)

0.022094140249759846
0.1037463976945245
0.4743035542747358
0.39985590778097985


In [13]:
# This piece of code is verifying that practices in each group are provided the same number of cases

solo_practice = []
small_practice = []
medium_practice = []
large_practice = []

for items in Practice_Case:
    if items[2] == 1:
        solo_practice.append(items[1])
    elif items[2] == 2:
        small_practice.append(items[1])
    elif items[2] == 3:
        medium_practice.append(items[1])
    elif items[2] == 4:
        large_practice.append(items[1])

print(solo_practice,small_practice,medium_practice,large_practice)
    

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4] [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25] [185, 185, 185, 185, 185, 185, 185, 185, 185]
